In [1]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [89]:
pip install syllables

     -------------------------------------- 939.3/939.3 kB 7.4 MB/s eta 0:00:00
  Using cached importlib_resources-5.12.0-py3-none-any.whl (36 kB)
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.11.3
    Uninstalling importlib-metadata-4.11.3:
      Successfully uninstalled importlib-metadata-4.11.3
Note: you may need to restart the kernel to use updated packages.


In [2]:
!python -m spacy download en_core_web_md

     ---------------------------------------- 42.8/42.8 MB 3.1 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [3]:
pip install textblob

Note: you may need to restart the kernel to use updated packages.


In [181]:
import pandas as pd
from bs4 import BeautifulSoup, NavigableString
import requests
import string
import spacy
import os
import nltk
# download necessary nltk data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')   
from textblob import TextBlob
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rahul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rahul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Rahul\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Rahul\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [182]:
# Read the input.xlsx file
data = pd.read_excel("C:\\Users\\Rahul\\Downloads\\Input.xlsx")
data# Create a folder for all the files

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...
...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...
110,147,https://insights.blackcoffer.com/the-future-of...
111,148,https://insights.blackcoffer.com/big-data-anal...
112,149,https://insights.blackcoffer.com/business-anal...


## Data Extraction

In [93]:
# Create a folder for all the files
os.makedirs('data', exist_ok=True)

In [183]:
import os
import requests
from bs4 import BeautifulSoup
from bs4.element import NavigableString
import pandas as pd

l = []
for url in data['URL']:
    title = url.split('/')[3]
    file_name = title + '.html'
    file_path = './data/' + file_name 
    #to scrape the data from the url and save it to the file if the file doesnt exist in the directory
    if not os.path.exists(file_path):
        #print('file doesnt exists')
        with open(file_path,'w') as C:
            r = requests.get(url, headers={"User-Agent": "XY"})
            htmlcontent = r.content.decode()
            C.write(htmlcontent)
    #to open each file and retrieve the beautifulsoup from the htmlcontent and put it in a dataframe
    with open(file_path, 'r', encoding='ISO-8859-1') as C:  
        htmlcontent = C.read()
    soup = BeautifulSoup(htmlcontent, 'html.parser')
    title = soup.title
    #article_Title=soup.find('div', attrs={'class': 'td-post-header'})
    article_content = soup.find('div', attrs={'class': 'td-post-content'})
    if article_content is None:
        article_text = None
    else:
        article_text = ''
        for element in article_content:
            if not isinstance(element, NavigableString):
                text = element.text
                article_text += text
    l.append(article_text)

df = pd.DataFrame(l, columns=['Text'])
df


,Text
0,IntroductionIf anything kills over 10 million...
1,"Human minds, a fascination in itself carrying ..."
2,IntroductionAI is rapidly evolving in the empl...
3,Anything that could give rise to smarter-than...
4,Machine intelligence is the last invention th...
...,...
109,Reconciling with the financial realities of an...
110,What Is an Investment?An investment is a resou...
111,Quality and affordable healthcare is a vision ...
112,Analytics is a statistical scientific process ...


### This code does the following:

* Uses the "requests" module to get the HTML content from the URL.
* Uses "BeautifulSoup" to parse the HTML content.
* Gets the title of the page from the "title" tag.
* Constructs a file name from the title and saves the HTML content to a file in the "data" directory.
* Opens the file and reads the HTML content.
* Finds the "div" tag with class "td-post-content" and extracts the article content from it.
* Appends the article content to a list.
* Constructs a DataFrame from the list and returns it.
#### Note that the code also handles exceptions when the URL is invalid or when the article content is not found in the HTML content.

## checking for null nvalues

In [184]:
df.isna().sum()

Text    12
dtype: int64

In [185]:
df.isnull().any(axis=1)
df.loc[null_rows]


,Text
5,None
7,None
11,None
20,None
47,None
50,None
59,None
80,None
83,None
103,None


In [197]:
rows_to_drop = [5, 7, 11, 20, 47, 50, 59, 80, 83, 103, 105, 107]
clean_df= data.drop(index=rows_to_drop)

In [198]:
clean_df

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...
...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...
110,147,https://insights.blackcoffer.com/the-future-of...
111,148,https://insights.blackcoffer.com/big-data-anal...
112,149,https://insights.blackcoffer.com/business-anal...


In [187]:
clean.shape

(102, 1)

In [193]:
clean= df.dropna()

In [194]:
clean.shape

(102, 1)

In [190]:
clean_df.isnull().sum()

Text    0
dtype: int64

In [210]:
clean_df

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...
...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...
110,147,https://insights.blackcoffer.com/the-future-of...
111,148,https://insights.blackcoffer.com/big-data-anal...
112,149,https://insights.blackcoffer.com/business-anal...


## Text Analysis

In [200]:
stop_words = set(stopwords.words('english'))

processed_text = []
for text in clean['Text']:
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stop words
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    # Join the filtered tokens to form a sentence
    processed_text.append(' '.join(filtered_tokens))

clean['Processed Text'] = processed_text


In [201]:
# create stemmer and lemmatizer objects
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# define function to preprocess text
def preprocess_text(text):
    # remove punctuation and convert to lowercase
    text = text.translate(str.maketrans('', '', string.punctuation)).lower()
    # tokenize text
    tokens = nltk.word_tokenize(text)
    # remove stopwords
    tokens = [token for token in tokens if token not in stopwords.words('english')]
    # perform stemming
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    # perform lemmatization
    lemma_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    # join tokens back into a string
    stemmed_text = ' '.join(stemmed_tokens)
    lemma_text = ' '.join(lemma_tokens)
    return stemmed_text, lemma_text

# apply preprocess_text function to text column and create new columns for stemmed and lemmatized text
clean['stemmed_text'], clean['lemma_text'] = zip(*clean['Text'].apply(preprocess_text))


In [202]:
clean

,Text,Processed Text,stemmed_text,lemma_text
0,IntroductionIf anything kills over 10 million...,IntroductionIf anything kills 10 million peop...,introductionif anyth kill 10 million peopl ne...,introductionif anything kill 10 million peopl...
1,"Human minds, a fascination in itself carrying ...","Human minds , fascination carrying potential t...",human mind fascin carri potenti tinker natur p...,human mind fascination carrying potential tink...
2,IntroductionAI is rapidly evolving in the empl...,IntroductionAI rapidly evolving employment sec...,introductionai rapidli evolv employ sector par...,introductionai rapidly evolving employment sec...
3,Anything that could give rise to smarter-than...,Anything could give rise smarter-than-human i...,anyth could give rise smarterthanhuman intell...,anything could give rise smarterthanhuman int...
4,Machine intelligence is the last invention th...,Machine intelligence last invention humanity ...,machin intellig last invent human ever need m...,machine intelligence last invention humanity ...
...,...,...,...,...
109,Reconciling with the financial realities of an...,Reconciling financial realities MBA education ...,reconcil financi realiti mba educ may come lat...,reconciling financial reality mba education ma...
110,What Is an Investment?An investment is a resou...,Investment ? investment resource thing procure...,investmentan invest resourc thing procur objec...,investmentan investment resource thing procure...
111,Quality and affordable healthcare is a vision ...,Quality affordable healthcare vision governmen...,qualiti afford healthcar vision govern across ...,quality affordable healthcare vision governmen...
112,Analytics is a statistical scientific process ...,Analytics statistical scientific process disco...,analyt statist scientif process discov present...,analytics statistical scientific process disco...


### POSITIVE SCORE

In [212]:
# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Calculate the positive score for each article
positive_scores = []
for text in clean['Processed Text']:
    # Calculate the sentiment scores using VADER
    sentiment_scores = analyzer.polarity_scores(text)
    # Extract the positive score
    positive_score = sentiment_scores['pos']
    positive_scores.append(positive_score)

# Add the positive scores to the DataFrame
clean_df['POSITIVE SCORE'] = positive_scores

### NEGATIVE SCORE

In [213]:
# Calculate the negative score for each article
negative_scores = []
for text in clean['Processed Text']:
    # Calculate the sentiment scores using VADER
    sentiment_scores = analyzer.polarity_scores(text)
    # Extract the negative score
    negative_score = sentiment_scores['neg']
    negative_scores.append(negative_score)

# Add the negative scores to the DataFrame
clean_df['NEGATIVE SCORE'] = negative_scores

### POLARITY SCORE

In [214]:
# Calculate the polarity score for each article
polarity_scores = []
for text in clean['Processed Text']:
    # Calculate the sentiment scores using VADER
    sentiment_scores = analyzer.polarity_scores(text)
    # Extract the positive and negative scores
    positive_score = sentiment_scores['pos']
    negative_score = sentiment_scores['neg']
    # Calculate the polarity score by subtracting the negative score from the positive score
    polarity_score = positive_score - negative_score
    polarity_scores.append(polarity_score)

# Add the polarity scores to the DataFrame
clean_df['POLARITY SCORE'] = polarity_scores

### SUBJECTIVITY SCORE

In [215]:
subjectivity_scores = []
for text in clean['Processed Text']:
    # Calculate the TextBlob sentiment scores
    sentiment_scores = TextBlob(text).sentiment
    # Extract the subjectivity score
    subjectivity_score = sentiment_scores.subjectivity
    subjectivity_scores.append(subjectivity_score)

# Add the subjectivity scores to the DataFrame
clean_df['SUBJECTIVITY SCORE'] = subjectivity_scores

### AVG SENTENCE LENGTH

In [216]:
# Calculate the average sentence length for each article
avg_sentence_lengths = []
for text in clean['Processed Text']:
    # Tokenize the text into sentences
    sentences = nltk.sent_tokenize(text)
    # Count the number of words in each sentence and calculate the average
    total_words = sum(len(nltk.word_tokenize(sentence)) for sentence in sentences)
    avg_sentence_length = total_words / len(sentences)
    avg_sentence_lengths.append(avg_sentence_length)

# Add the average sentence lengths to the DataFrame
clean_df['AVG SENTENCE LENGTH'] = avg_sentence_lengths

### PERCENTAGE OF COMPLEX WORDS

In [217]:
import syllables

def count_syllables(word):
    return syllables.estimate(word)

def calculate_complex_word_percentage(text):
    # Tokenize the text into individual words
    words = word_tokenize(text)
    # Count the number of words with more than 2 syllables
    num_complex_words = 0
    for word in words:
        if count_syllables(word) > 2:
            num_complex_words += 1
    # Calculate the percentage of complex words
    complex_word_percentage = (num_complex_words / len(words)) * 100
    return complex_word_percentage

# Calculate the percentage of complex words for each article
complex_word_percentages = []
for text in clean['Processed Text']:
    complex_word_percentage = calculate_complex_word_percentage(text)
    complex_word_percentages.append(complex_word_percentage)
    
# Add the complex word percentages to the DataFrame
clean_df['PERCENTAGE OF COMPLEX WORDS'] = complex_word_percentages


### FOG INDEX

In [218]:
# Define a function to calculate the FOG Index for a given text
def calculate_fog_index(text):
    # Tokenize the text into sentences
    sentences = sent_tokenize(text)
    # Calculate the average sentence length
    sentence_lengths = [len(word_tokenize(sentence)) for sentence in sentences]
    avg_sentence_length = sum(sentence_lengths) / len(sentence_lengths)
    # Calculate the percentage of complex words
    stopwords_list = set(stopwords.words('english'))
    words = word_tokenize(text.lower())
    complex_words = [word for word in words if len(word) > 2 and word not in stopwords_list]
    percentage_complex_words = len(complex_words) / len(words)
    # Calculate the FOG Index
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)
    return fog_index

# Calculate the FOG Index for each article
fog_scores = []
for text in clean['Processed Text']:
    fog_score = calculate_fog_index(text)
    fog_scores.append(fog_score)

# Add the FOG Index scores to the DataFrame
clean_df['FOG INDEX'] = fog_scores


### AVG NUMBER OF WORDS PER SENTENCE

In [219]:
# Calculate the average number of words per sentence for each article
avg_words_per_sentence = []
for text in clean['Text']:
    # Tokenize the text into sentences
    sentences = nltk.sent_tokenize(text)
    # Calculate the average number of words per sentence
    num_words = sum(len(nltk.word_tokenize(sentence)) for sentence in sentences)
    avg_words_per_sentence.append(num_words / len(sentences))

# Add the average number of words per sentence to the DataFrame
clean_df['AVG NUMBER OF WORDS PER SENTENCE'] = avg_words_per_sentence

### COMPLEX WORD COUNT

In [220]:
# Function to check if a word is a complex word
def is_complex_word(word):
    synsets = wordnet.synsets(word)
    if len(synsets) == 0:
        return False
    else:
        synonyms = set()
        for synset in synsets:
            synonyms.update([lemma.name() for lemma in synset.lemmas()])
        if len(synonyms) > 1:
            return True
        else:
            return False

# Calculate the complex word count for each article
complex_word_counts = []
for text in clean['Processed Text']:
    # Split the text into words
    words = nltk.word_tokenize(text)
    # Count the number of complex words
    complex_word_count = sum([1 for word in words if is_complex_word(word)])
    complex_word_counts.append(complex_word_count)

# Add the complex word counts to the DataFrame
clean_df['COMPLEX WORD COUNT'] = complex_word_counts

### WORD COUNT

In [221]:
word_counts = []
for text in clean['Processed Text']:
    # Split the text into words
    words = text.split()
    # Count the number of words
    count = len(words)
    word_counts.append(count)

# Add the word counts to the DataFrame
clean_df['WORD COUNT'] = word_counts

### SYLLABLE PER WORD

In [222]:
import nltk
from nltk.corpus import cmudict
nltk.download('cmudict')

# Initialize the CMU pronunciation dictionary
cmu_dict = cmudict.dict()

# Function to count the number of syllables in a word
def count_syllables(word):
    # Use the CMU pronunciation dictionary to count the number of syllables
    if word.lower() in cmu_dict:
        return len([syl for syl in cmu_dict[word.lower()][0] if str.isdigit(syl[-1])])
    else:
        return 0

# Calculate the average number of syllables per word for each article
syllables_per_word = []
for text in clean['Processed Text']:
    words = text.split()
    # Calculate the total number of syllables in the article
    total_syllables = sum([count_syllables(word) for word in words])
    # Calculate the total number of words in the article
    total_words = len(words)
    # Calculate the average number of syllables per word
    if total_words > 0:
        avg_syllables_per_word = total_syllables / total_words
    else:
        avg_syllables_per_word = 0
    syllables_per_word.append(avg_syllables_per_word)

# Add the syllables per word to the DataFrame
clean_df['SYLLABLE PER WORD'] = syllables_per_word

[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\Rahul\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


### PERSONAL PRONOUNS

In [223]:
import nltk

# Download the necessary NLTK data
nltk.download('averaged_perceptron_tagger')

# Define a function to count personal pronouns
def count_personal_pronouns(text):
    """
    This function takes a string of text and returns the count of personal pronouns (first, second, and third person).
    """
    # Tokenize the text into words
    words = nltk.word_tokenize(text)
    # Tag the words with part-of-speech tags
    tagged_words = nltk.pos_tag(words)
    # Count the personal pronouns
    personal_pronoun_count = 0
    for word, tag in tagged_words:
        if tag == 'PRP' or tag == 'PRP$':
            personal_pronoun_count += 1
    return personal_pronoun_count

# Calculate the count of personal pronouns for each article
personal_pronoun_counts = []
for text in clean['Processed Text']:
    personal_pronoun_count = count_personal_pronouns(text)
    personal_pronoun_counts.append(personal_pronoun_count)

# Add the personal pronoun counts to the DataFrame
clean_df['PERSONAL PRONOUNS'] = personal_pronoun_counts

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Rahul\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


### AVG WORD LENGTH

In [224]:
# Calculate the average word length for each article
avg_word_lengths = []
for text in clean['Processed Text']:
    # Tokenize the text into words
    words = nltk.word_tokenize(text)
    # Calculate the total length of all words
    total_word_length = sum(len(word) for word in words)
    # Calculate the average word length
    avg_word_length = total_word_length / len(words)
    avg_word_lengths.append(avg_word_length)

# Add the average word lengths to the DataFrame
clean_df['AVG WORD LENGTH'] = avg_word_lengths

In [227]:
clean_df

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37,https://insights.blackcoffer.com/ai-in-healthc...,0.166,0.057,0.109,0.459874,23.250000,40.168971,9.634869,36.622642,944,1301,1.801691,1,6.546851
1,38,https://insights.blackcoffer.com/what-if-the-c...,0.201,0.092,0.109,0.404217,13.614286,28.436516,5.755053,23.308824,659,952,1.536765,4,5.486884
2,39,https://insights.blackcoffer.com/what-jobs-wil...,0.153,0.079,0.074,0.477847,17.447761,41.745081,7.311354,27.761194,864,1169,1.863131,2,6.422583
3,40,https://insights.blackcoffer.com/will-machine-...,0.228,0.059,0.169,0.491477,13.423077,31.136581,5.693204,22.782051,770,1047,1.653295,2,5.741165
4,41,https://insights.blackcoffer.com/will-ai-repla...,0.197,0.061,0.136,0.501298,16.178082,32.853514,6.796720,28.367647,836,1177,1.672048,7,5.927180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...,0.160,0.071,0.089,0.399679,15.743590,36.319218,6.644016,25.684211,416,614,1.612378,1,6.596091
110,147,https://insights.blackcoffer.com/the-future-of...,0.134,0.025,0.109,0.393396,23.916667,30.749129,9.883740,35.469388,789,1148,1.533972,0,5.781359
111,148,https://insights.blackcoffer.com/big-data-anal...,0.133,0.133,0.000,0.395841,13.131148,37.453184,5.593033,20.721311,588,801,1.799001,0,6.041199
112,149,https://insights.blackcoffer.com/business-anal...,0.262,0.007,0.255,0.541296,26.444444,47.478992,10.920635,42.166667,342,476,1.941176,0,7.029412


In [228]:
clean_df.to_csv('output.csv', index=False)
